In [2]:
import pandas as pd
from netCDF4 import Dataset
import xarray as xr
import h5py
import numpy as np
import os
import os.path
import copy
import warnings
from siphon import catalog
from sklearn.tree import DecisionTreeRegressor
from dask.distributed import Client, LocalCluster
overwrite = False

warnings.filterwarnings(
    "ignore",
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated",
    category=FutureWarning,
)

In [3]:
experiments = [
               '1pctCO2', 'abrupt-4xCO2', 'historical', # CMIP
               'hist-GHG', 'hist-aer', # DAMIP
               'ssp370', 'ssp370-lowNTCF', 'ssp585' # ScenarioMIP
]

In [4]:
def forge_output(experiment) -> np.array:
    """
    Creates the output datasets for tas, pr, and pr_perc respectively out of the files in the associated filepaths. Returns them in one larger transposed array.
    """
    tas_prior = (xr.open_dataset(f'data_store/tas_means/NorESM2-LM_{experiment}_tas.nc')).sel(member='r1i1p1f1')['tas'] # TAS
    pr_prior = (xr.open_dataset(f'data_store/pr_means/NorESM2-LM_{experiment}_pr.nc')).sel(member='r1i1p1f1')['pr'] # PR
    pr_perc_prior = (xr.open_dataset(f'data_store/pr_perc/NorESM2-LM_{experiment}_pr_p90_grid_annual_total_mm.nc')).sel(member='r1i1p1f1')['pr_p90_grid_annual_total_mm'] # PR_PERC

    tas_al, pr_al, prp90_al = xr.align(tas_prior, pr_prior, pr_perc_prior, join='inner') # Aligning the datasets on year    
    toreturn = xr.Dataset({"tas": tas_al, "pr": pr_al, "pr_p90": prp90_al}) # Combining the datasets
    
    return (np.array(toreturn.to_dataarray())).T

def forge_input(experiment) -> np.array:
    """
    Creates the input dataset out of the preprocessed inputs stored in input_data
    """
    input_prior = xr.open_dataset(f'input_data/inputs_{experiment}.nc')
    
    try: # If neccessary, scales the inputs by global weights
        weights = np.cos(np.deg2rad(input_prior['latitude'])).rename('weights')
        SO2_gm = input_prior['SO2'].weighted(weights).mean(['latitude','longitude'], skipna=True)
        BC_gm  = input_prior['BC'] .weighted(weights).mean(['latitude','longitude'], skipna=True)
        input_prior = input_prior.drop_vars(['SO2','BC']).assign(SO2=SO2_gm, BC=BC_gm)
        return (np.array(input_prior.to_dataarray())).T
    except: # Otherwise, just returns
        return (np.array(input_prior.to_dataarray())).T

def mean_first3_available(da):
    """
    For experiments with multiple members, takes the average over the 3 members and returns a singular value. 
    """
    if "member" not in da.dims:
        return da
    avail = da["member"].astype(str).values  
    sel = [m for m in MEMBERS3 if m in avail]
    if sel:
        return da.sel(member=sel, drop=True).mean("member", skipna=True)
    # none of the named members exist → take first up to 3
    return da.isel(member=slice(0, 3)).mean("member", skipna=True)

In [5]:
# NOTE: This is to fix a particular issue where input were in delta change of TAS, PR, and PR_PERC, while output were in total TAS, PR, and PR_PERC. 
# To remedy this, we adjust the inputs by the historical means of the last 50 years of the historical dataset.

fifty_year_close = forge_output("historical")[-50:]
fifty_year_close_avgs = fifty_year_close.mean(axis=0)

hist_tas_adj = fifty_year_close_avgs[0]
hist_pr_adj = fifty_year_close_avgs[1]
hist_pr90_adj = fifty_year_close_avgs[2]

In [6]:
MODEL = "NorESM2-LM"  # Climate model name
DATA_BASE = "data_store"  # Base directory for stored data
MEMBERS3 = ["r1i1p1f1", "r2i1p1f1", "r3i1p1f1"]  # Preferred member IDs

# build outputs

cols = ["tas", "pr", "pr_p90"]  # Output variable names
full_output = pd.DataFrame(columns=cols + ["experiment", "year"])  # Container for all outputs

for experiment in experiments:
    # Load full member fields
    tas = xr.open_dataset(f"{DATA_BASE}/tas_means/{MODEL}_{experiment}_tas.nc")["tas"]  # TAS field
    pr  = xr.open_dataset(f"{DATA_BASE}/pr_means/{MODEL}_{experiment}_pr.nc")["pr"]     # PR field
    p90 = xr.open_dataset(  # 90th percentile PR field
        f"{DATA_BASE}/pr_perc/{MODEL}_{experiment}_pr_p90_grid_annual_total_mm.nc"
    )["pr_p90_grid_annual_total_mm"]

    # Average over up to the first 3 available members
    tas = mean_first3_available(tas)  # Member-mean TAS
    pr  = mean_first3_available(pr)   # Member-mean PR
    p90 = mean_first3_available(p90)  # Member-mean PR90

    # Align on common time/year dimension
    tas_al, pr_al, p90_al = xr.align(tas, pr, p90, join="inner")  # Align member-mean fields
    coord = "year" if "year" in tas_al.coords else "time"         # Handle different coordinate naming
    years_out = tas_al[coord].values.astype(int)                  # Extract time coordinate as int

    # Stack aligned member-mean fields into a dataset/array
    ds_out = xr.Dataset({"tas": tas_al, "pr": pr_al, "pr_p90": p90_al})  # Combined member-mean dataset
    arr = np.array(ds_out.to_dataarray()).T  # (n_years, 3) array of tas/pr/pr_p90

    # Create DataFrame for this experiment
    df = pd.DataFrame(arr, columns=cols)  # Per-experiment outputs
    df["experiment"] = experiment        # Tag with experiment name
    df["year"] = years_out              # Attach aligned years

    # Append to full output container
    full_output = pd.concat([full_output, df], ignore_index=True)

# build inputs

cols = ["CO2", "SO2", "CH4", "BC"]  # Input variable names
full_input = pd.DataFrame(columns=cols + ["experiment", "year"])  # Container for all inputs

for experiment in experiments:
    arr = forge_input(experiment)  # shape: (n_years_in, 4) input array for this experiment

    cand1 = os.path.join("input_data", f"inputs_{experiment}.nc")  # Experiment-specific input file
    cand2 = os.path.join("input_data", "inputs_ssp370.nc")         # Fallback input file
    in_path = cand1 if os.path.exists(cand1) else cand2            # Choose available file
    ds_in = xr.open_dataset(in_path)  # Open input dataset

    years_in = ds_in["time"].values.astype(int)  # Input time coordinate as int

    df = pd.DataFrame(arr, columns=cols)  # Per-experiment inputs
    df["experiment"] = experiment         # Tag with experiment name
    df["year"] = years_in                 # Attach input years

    years_out_set = set(  # Years present in outputs for this experiment
        full_output.loc[full_output["experiment"] == experiment, "year"].astype(int)
    )
    df = df[df["year"].isin(years_out_set)]  # Keep only years with matching outputs

    # Append to full input container
    full_input = pd.concat([full_input, df], ignore_index=True)

# ensure common (experiment, year) keys across inputs and outputs

keys = (
    full_output[["experiment", "year"]]
    .merge(full_input[["experiment", "year"]], on=["experiment", "year"], how="inner")  # Common keys
    .drop_duplicates()  # Remove duplicate key rows
)

full_output = keys.merge(full_output, on=["experiment", "year"], how="left")  # Reindex outputs
full_input  = keys.merge(full_input,  on=["experiment", "year"], how="left")  # Reindex inputs

# sanity checks on alignment

print(
    "checking for experiment alignment: ",
    (full_input["experiment"] == full_output["experiment"]).mean(),  # Share of matching experiments
)
print(
    "checking for year alignment: ",
    (full_input["year"] == full_output["year"]).mean(),  # Share of matching years
)

# drop key columns now that alignment is guaranteed

full_output.drop(columns=["experiment", "year"], inplace=True)
full_input.drop(columns=["experiment", "year"], inplace=True)

print("full_output:", full_output.shape)  # Final output array shape
print("full_input :", full_input.shape)   # Final input array shape


checking for experiment alignment:  1.0
checking for year alignment:  1.0
full_output: (1344, 3)
full_input : (1344, 4)


In [7]:
### BUILDING MODELS

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [9]:
test_inputs = xr.open_dataset("/glade/u/home/nchu/test_data/inputs_ssp245.nc")  # Load test inputs from preprocessed data
weights = np.cos(np.deg2rad(test_inputs['latitude'])).rename('weights')        # Latitude-based area weights
SO2_gm = test_inputs['SO2'].weighted(weights).mean(['latitude', 'longitude'], skipna=True)  # Area-weighted mean SO2
BC_gm  = test_inputs['BC'] .weighted(weights).mean(['latitude', 'longitude'], skipna=True)  # Area-weighted mean BC
test_inputs = (  # Replace gridded SO2/BC with their global means
    test_inputs.drop_vars(['SO2', 'BC']).assign(SO2=SO2_gm, BC=BC_gm)
)
test_inputs = (np.array(test_inputs.to_dataarray())).T  # Convert to transposed NumPy array to match forged inputs/outputs
print(test_inputs.shape)  # Check input array shape

test_outputs = xr.open_dataset(  # Load test outputs from withheld output experiment (ssp245)
    "/glade/u/home/nchu/test_data/outputs_ssp245.nc"
).sel(member=1, drop=True)  # Select single ensemble member
test_outputs = test_outputs.drop_vars("diurnal_temperature_range")  # Drop unused variable (for our replication)
weights_out = np.cos(np.deg2rad(test_outputs['lat'])).rename('weights')  # Latitude-based area weights for outputs

tas_gm  = test_outputs['tas'] .weighted(weights_out).mean(['lat', 'lon'], skipna=True)  # Area-weighted mean TAS
pr_gm   = test_outputs['pr']  .weighted(weights_out).mean(['lat', 'lon'], skipna=True)  # Area-weighted mean PR
pr90_gm = test_outputs['pr90'].weighted(weights_out).mean(['lat', 'lon'], skipna=True)  # Area-weighted mean PR90

test_outputs = (  # Replace gridded tas/pr/pr90 with global means
    test_outputs.drop_vars(['tas', 'pr', 'pr90']).assign(tas=tas_gm, pr=pr_gm, pr90=pr90_gm)
)
test_outputs = test_outputs[['tas', 'pr', 'pr90']]  # Keep only target variables in defined order
test_output = (np.array(test_outputs.to_dataarray())).T  # Convert to transposed NumPy array (n_years × 3)

test_output[:, 0] += hist_tas_adj   # Add historical TAS adjustment
test_output[:, 1] += hist_pr_adj    # Add historical PR adjustment
test_output[:, 2] += hist_pr90_adj  # Add historical PR90 adjustment

print(test_output.shape)  # Check output array shape


(86, 4)
(86, 3)


In [10]:
### Random Forest Model

In [11]:
# Keeping it simple, to be more replicative of the paper.

rf = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    max_features="sqrt",
    min_samples_leaf=2,
    n_jobs=-2,
    random_state=42
)
rf.fit(full_input, full_output)

,n_estimators,600
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [12]:
pred_rf = rf.predict(test_inputs) # making predictions with the model

/glade/u/home/nchu/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [25]:
# copy so we don't mutate the originals
pred_rf_conv = pred_rf.copy()
test_out_conv = test_output.copy()

# convert precipitation from kg m-2 s-1 to mm/day
pred_rf_conv[:, 1] *= 86400.0   
test_out_conv[:, 1] *= 86400.0  

err = pred_rf_conv - test_out_conv                     # calculating error
num = np.mean(err**2, axis=0)                          # MSE per target
den = np.abs(np.mean(test_out_conv, axis=0))           # |mean(y)| per target

rmse = np.sqrt(num)                                    # RMSE
nrmse = np.sqrt(num / den)                             # ClimateBench-style NRMSE_x

per_target = pd.DataFrame(
    {"RF_NRMSE": nrmse, "RF_RMSE": rmse},
    index=["tas", "pr", "pr90"]
)
print(per_target)

print("Overall Random Forest Global NRMSE (avg):", nrmse.mean())
print("Overall Random Forest RMSE (avg):", rmse.mean())


      RF_NRMSE    RF_RMSE
tas   0.025693   0.437149
pr    0.014620   0.024835
pr90  0.493563  20.620191
Overall Random Forest Global NRMSE (avg): 0.17795889487226027
Overall Random Forest RMSE (avg): 7.027391438879795


In [27]:
err = pred_rf - test_output # calculating error                   
num = np.mean(err**2, axis=0) # calculating squared mean error              
den = np.abs(np.mean(test_output, axis=0)) # calculating test output absolute mean

rmse = np.sqrt(num) # calculating rmse
nrmse = np.sqrt(num / den) # calculating nrmse        

per_target = pd.DataFrame(
    {"RF_NRMSE": nrmse, "RF_RMSE": rmse},
    index=["tas", "pr", "pr90"]
)
print(per_target)

print("Overall Random Forest NRMSE (avg):", nrmse.mean())
print("Overall Random Forest RMSE (avg):", rmse.mean())


      RF_NRMSE       RF_RMSE
tas   0.025693  4.371493e-01
pr    0.000050  2.874368e-07
pr90  0.493563  2.062019e+01
Overall Random Forest Global NRMSE (avg): 0.17310198192296847
Overall Random Forest RMSE (avg): 7.019113355625836


In [28]:
### Gaussian Process Model

In [29]:
from esem import gp_model
from esem.data_processors import Whiten, Normalise

In [30]:
# processing the inputs for the Gaussian Process model

gp_inputs = np.array(full_input) 
data_min = np.min(gp_inputs)
data_max = np.max(gp_inputs)
normal_inputs = (gp_inputs - data_min) / (data_max - data_min)

gp_test_inputs = np.array(test_inputs)
data_min = np.min(gp_test_inputs)
data_max = np.max(gp_test_inputs)
normal_test_inputs = (gp_test_inputs - data_min) / (data_max - data_min)

In [31]:
# the esem library uses the following approach for model usage:

tas_gpr = gp_model(normal_inputs, np.array(full_output['tas']))
pr_gpr = gp_model(normal_inputs, np.array(full_output['pr']))
pr90_gpr = gp_model(normal_inputs, np.array(full_output['pr_p90']))

In [32]:
# training each of the models, one for each target variable

tas_gpr.train()
pr_gpr.train()
pr90_gpr.train()

In [33]:
# using the same esem library, we make predictions for each of the target variables

tas_pred_gpr = tas_gpr.predict(normal_test_inputs)[0]
pr_pred_gpr = pr_gpr.predict(normal_test_inputs)[0]
pr90_pred_gpr = pr90_gpr.predict(normal_test_inputs)[0]
pred_gpr = np.stack([tas_pred_gpr, pr_pred_gpr, pr90_pred_gpr], axis=1)

In [34]:
# copy so we don't mutate the originals
pred_gp_conv = pred_gpr.copy()
test_out_conv = test_output.copy()

# convert precipitation from kg m-2 s-1 to mm/day
pred_gp_conv[:, 1] *= 86400.0   
test_out_conv[:, 1] *= 86400.0  

err = pred_gp_conv - test_out_conv # calculating error                   
num = np.mean(err**2, axis=0) # calculating squared mean error              
den = np.abs(np.mean(test_out_conv, axis=0)) # calculating test output absolute mean

rmse = np.sqrt(num) # calculating rmse
nrmse = np.sqrt(num / den) # calculating nrmse        

per_target = pd.DataFrame(
    {"GP_NRMSE": nrmse, "GP_RMSE": rmse},
    index=["tas", "pr", "pr90"]
)
print(per_target)

print("Overall Gaussian Process NRMSE (avg):", nrmse.mean())
print("Overall Gaussian Process RMSE (avg):", rmse.mean())


      GP_NRMSE     GP_RMSE
tas   0.076068    1.294241
pr    1.073115    1.822807
pr90  8.685321  362.857303
Overall Gaussian Process Global NRMSE (avg): 3.278167919096672
Overall Gaussian Process RMSE (avg): 121.99145008437438


In [98]:
### Neural Network

In [99]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense, Activation, Conv2D, Flatten, Input, Reshape, AveragePooling2D,
    MaxPooling2D, Conv2DTranspose, TimeDistributed, LSTM, GlobalAveragePooling2D,
    BatchNormalization
)
from tensorflow.keras.regularizers import l2

import random 
seed = 6 
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


In [102]:
# Use the FINAL aligned full_input / full_output
X_raw = full_input.to_numpy().astype("float32")        # (N, 4): CO2, SO2, CH4, BC
y_raw = full_output.to_numpy().astype("float32")       # (N, 3): tas, pr, pr90

print("X_raw shape:", X_raw.shape)
print("y_raw shape:", y_raw.shape)
assert X_raw.shape[0] == y_raw.shape[0]

# Scale inputs per feature
X_min = X_raw.min(axis=0, keepdims=True)
X_max = X_raw.max(axis=0, keepdims=True)
X = (X_raw - X_min) / (X_max - X_min + 1e-12)

# Scale outputs per feature
y_mean = y_raw.mean(axis=0, keepdims=True)
y_std  = y_raw.std(axis=0, keepdims=True) + 1e-12
y = (y_raw - y_mean) / y_std

model = keras.Sequential([
    keras.layers.Input(shape=(X.shape[1],)),   # 4 features
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(3, activation="linear"),  # scaled tas, pr, pr90
])

model.compile(optimizer="rmsprop", loss="mse", metrics=["mse"])
model.summary()


X_raw shape: (1344, 4)
y_raw shape: (1344, 3)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,675 (18.26 KB)

 Trainable params: 4,675 (18.26 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:
history = model.fit(
    X,
    y,                  # scaled targets
    batch_size=128,
    epochs=30,
    validation_split=0.2,
    shuffle=True,
    verbose=1,
)


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.9770 - mse: 0.9770 - val_loss: 0.6181 - val_mse: 0.6181
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8622 - mse: 0.8622 - val_loss: 0.5378 - val_mse: 0.5378
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7595 - mse: 0.7595 - val_loss: 0.5260 - val_mse: 0.5260
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6674 - mse: 0.6674 - val_loss: 0.6027 - val_mse: 0.6027
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5958 - mse: 0.5958 - val_loss: 0.7200 - val_mse: 0.7200
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5449 - mse: 0.5449 - val_loss: 0.7986 - val_mse: 0.7986
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5113 - mse: 0.5113 - val_loss: 0.8070 - val_mse: 0.8070
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4894 - mse: 0.4894 - val_loss: 0.7646 - val_mse: 0.7646
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4743 - mse: 0.4743 - val_loss

In [106]:
# Scale using TRAIN min/max
X_test = (test_inputs - X_min) / (X_max - X_min + 1e-12)

# Predict scaled outputs
pred_scaled = model.predict(X_test)

# Unscale predictions back to original units
pred_nn = pred_scaled * y_std + y_mean      


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [107]:
# copy so we don't mutate the originals
pred_nn_conv = pred_nn.copy()
test_out_conv = test_output.copy()  

# convert precipitation from kg m-2 s-1 to mm/day
sec_per_day = 86400.0
pred_nn_conv[:, 1] *= sec_per_day
test_out_conv[:, 1] *= sec_per_day

# calculate errors and metrics
err = pred_nn_conv - test_out_conv                      # error
num = np.mean(err**2, axis=0)                           # MSE per target (in physical units)
den = np.abs(np.mean(test_out_conv, axis=0))            # |mean(y)| per target

rmse = np.sqrt(num)                                     # RMSE
nrmse = np.sqrt(num / den)                              # ClimateBench-style NRMSE_x

per_target = pd.DataFrame(
    {"NN_NRMSE": nrmse, "NN_RMSE": rmse},
    index=["tas", "pr", "pr90"]
)
print(per_target)

print("Overall Neural Network NRMSE (avg):", nrmse.mean())
print("Overall Neural Network RMSE (avg):", rmse.mean())


      NN_NRMSE    NN_RMSE
tas   0.026113   0.444295
pr    0.024649   0.041870
pr90  1.719408  71.833833
Overall Neural Network NRMSE (avg): 0.5900570055783784
Overall Neural Network RMSE (avg): 24.10666614120518
